In [1]:
import numpy as np
import pandas as pd
from fastai.tabular import *

from pathlib import Path

In [2]:
from geopy import distance

In [3]:
data_pth = Path('./data')

In [4]:
def get_date_stuff(dt):
    return pd.Series(
        data = {
            'Year': dt.year - 2009,
            'Month': dt.month,
            'Day': dt.day,
            'Hour': dt.hour,
            'Quarter': dt.quarter
        }, 
        dtype = 'int')

In [ ]:
dtypes = {
    'fare_amount': 'float32',
    'pickup_datetime': 'str',
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'dropoff_longitude': 'float32',
    'dropoff_latitude': 'float32',
    'passenger_count': 'int16'
}
use_cols = [
    'fare_amount',
    'pickup_datetime',
    'pickup_longitude',
    'pickup_latitude', 
    'dropoff_longitude', 
    'dropoff_latitude',
    'passenger_count'
]
date_cols = ['pickup_datetime']
df = pd.read_csv(data_pth/'train.csv', nrows=None, 
                 dtype=dtypes, usecols=use_cols, 
                 parse_dates=date_cols, infer_datetime_format=True)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df = df.dropna(how='any')

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df = df.sort_values('pickup_datetime', ascending=True, ignore_index=True)

In [5]:
lat_long_cols = [
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude'
]

In [ ]:
df = df.merge(df.pickup_datetime.apply(get_date_stuff), left_index=True, right_index=True)

In [9]:
df['crow_distance'] = df[lat_long_cols].apply(lambda d: distance.distance(
                                    (d['pickup_latitude'], d['pickup_longitude']),
                                    (d['dropoff_latitude'], d['dropoff_longitude'])).km, axis=1)

/home/alastair/Documents/Development/ds/google-cloud-taxis/.venv/lib/python3.7/site-packages/geopy/point.py:405: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


ValueError: Latitude must be in the [-90; 90] range.

In [20]:
lat_cols = [
    'pickup_latitude',
    'dropoff_latitude'
]
long_cols = [
    'pickup_longitude',
    'dropoff_longitude'
]
def check_latlongs(d):
    if np.abs(d[lat_cols]) > 90.:
        return True
    elif np.abs(d[long_cols]) > 180.:
        return True
    else:
        return False

In [21]:
df['lat_long_wrong'] = False

In [23]:
df['lat_long_wrong'] = df[lat_long_cols].apply(check_latlongs, axis=1)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df.head()

In [ ]:
df.to_feather(data_pth/'checkpoints/train_data_no_nans.feather')

In [6]:
df = pd.read_feather(data_pth/'checkpoints/train_data_no_nans.feather')

In [7]:
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,30.200001,2009-01-01 00:00:27,-73.782104,40.644882,-73.963562,40.676350,1
1,15.000000,2009-01-01 00:00:46,-73.953735,40.806763,-73.989426,40.769543,1
2,4.200000,2009-01-01 00:00:49,-73.993187,40.727840,-73.990784,40.730007,1
3,5.800000,2009-01-01 00:01:04,-73.995132,40.734112,-73.998230,40.722874,2
4,14.600000,2009-01-01 00:01:04,-73.972481,40.742744,-73.918938,40.764496,1


In [ ]:
df = df.drop(['pickup_latitude', 'pickup_longitude'], axis=1)